In [1]:
def get_song_segment_key(song_id, segment_id):
    return "song_%s_%d" % (song_id, segment_id)

def get_song_id_from_segment_key(segment_key):
    return segment_key.split("_")[1]

song_segments={}
song_segments_array={}


In [2]:
from lib import hdf5_getters as hfg
import os
import csv
import string
import numpy as np
# Provide the starting root dir
# This is different for your particular setup!

indir = "./MillionSongSubset/data"

cnt = 0
limit = 100000000
import timeit
max_segment_size = 0
start = timeit.default_timer()
# Open the CSV file we will write to
songs  = None
songs = None
song_timbre = []
names = []
segments = {}
with open("output.csv", 'wb') as csvfile:
    # Column headers
    # Currently only 4 features being extracted
    # We can add as many as we want, just seperate with commas

    # Recursively visit each sub-dir till we reach the h5 files
    # Strip punctuation from features that are strings
    for root, dirs, filenames in os.walk(indir):
        for f in filenames:
            #print(os.path.join(root, f))

            # Use the hd5 wrappers to open the h5 file
            h5 = hfg.open_h5_file_read(os.path.join(root, f))

            #getting total number of songs in the h5 file
            total_songs = hfg.get_num_songs(h5)
            for songidx in range(total_songs):
                artist_name = hfg.get_artist_name(h5,songidx)
                artist = artist_name.decode('utf-8').translate(string.punctuation)

                # Get the title of the song
                title_song = hfg.get_title(h5,songidx)
                title = title_song.decode('utf-8').translate(string.punctuation)

                # Get release
                release_song = hfg.get_release(h5,songidx)
                release = release_song.decode('utf-8').translate(string.punctuation)
                
                #getting segments
                segments_start = hfg.get_segments_start(h5, songidx)
                segments_pitches = hfg.get_segments_pitches(h5, songidx)
                segments_timbre = hfg.get_segments_timbre(h5, songidx)
                segments_loudness_start = hfg.get_segments_loudness_start(h5, songidx)
                segments_loudness_max = hfg.get_segments_loudness_max(h5, songidx)
                track_id = hfg.get_track_id(h5, songidx)
                song_id = hfg.get_song_id(h5, songidx)
                N = segments_start.shape[0]
                segment_array=[]
                for i in range(N):
                    segment = np.array([])
                    segment = np.concatenate((segment, segments_pitches[i]), axis=0)
                    segment = np.concatenate((segment, segments_timbre[i]), axis=0)
                    segment = np.concatenate((segment, np.array([segments_loudness_start[i]])), axis=0)
                    segment = np.concatenate((segment, np.array([segments_loudness_max[i]])), axis=0)
                    segment_key = get_song_segment_key(song_id, i)
                    song_segments[segment_key] = segment
                    segment_array.append(segment)
                song_segments_array[song_id] = segment_array
                segments_bars_start = hfg.get_bars_start(h5, songidx)
                # Get duration
                duration = hfg.get_duration(h5,songidx)

                N = segments_start.shape[0]


                #segments_start = segments_start.reshape(N,1)
                #song_segment_data = np.stack(segments_start,segments_pitches)
                #print(song_segment_data.shape)
                #song = np.array([segments_start,segments_pitches,segments_timbre, \
                #                segments_loudness_start, segments_loudness_max, \
                #                segments_bars_start],dtype=object)
                #songs = np.append(songs, song)
                
                #print(segments_timbre.shape)
                song_timbre.append(segments_timbre)
                
                #########################################################

                # Print the current song and arists:
                max_segment_size = max(max_segment_size, len(segments_start))
                print(title + " by " + artist + " duration: " ,duration/60.0," minutes "," segments:",len(segments_start)," songid ",song_id,"trackid ",track_id)
                # print str(num_songs)
                # Move on to the next h5 file
                print("")
                print("")
                cnt+=1
                if cnt==limit:
                    break

            
            # Close the h5 file
            h5.close()
            if cnt==limit:
                break
        if cnt==limit:
            break
stop = timeit.default_timer()
print("Total songs: ",cnt,"total time: ",stop-start," max segment: ",max_segment_size)
print("Total song segments ",len(song_segments))


True Fuschnick by FU-Schnickens duration:  3.965380166666667  minutes   segments: 1239  songid  b'SONFOEG12A8C13B8B8' trackid  b'TRBBBMT128F42A0A4E'


Sleepy Lad by Maximilian Hecker duration:  2.9039379999999997  minutes   segments: 459  songid  b'SOFWWIA12AB01884A1' trackid  b'TRBBBEA128F93391BA'


Mother_ Mother_ Mother_ Pin A Rose On Me by Kitty Kallen duration:  2.689734  minutes   segments: 512  songid  b'SOQPFGN12A8C13C1B3' trackid  b'TRBBBVC128F429AB66'


Exodus by Sunscreem duration:  3.8739516666666662  minutes   segments: 1045  songid  b'SOICLQB12A8C13637C' trackid  b'TRBBBLA128F424E963'


Sickness by Agnostic Front duration:  1.5364278333333334  minutes   segments: 304  songid  b'SOPWWAV12A8C13A825' trackid  b'TRBBBOA128F42624DA'


Sea of Green by Slackbaba duration:  7.760536666666667  minutes   segments: 2051  songid  b'SOCWFVT12AB01890F8' trackid  b'TRBBBKS12903CDFCF4'


My Plastic Heart (Plastic Operator Remix) by Frost duration:  5.6041285  minutes   segments: 1558  so

Leo_ Are You Still Jumping Out Of Windows In Expensive Clothes? by My Little Airport duration:  2.0950128333333335  minutes   segments: 414  songid  b'SOFFUHF12AC4689476' trackid  b'TRBBOPX12903D106F7'


Creation by The James Taylor Quartet duration:  5.252346166666666  minutes   segments: 997  songid  b'SOTGNGT12A8C142826' trackid  b'TRBBOUO128F92D246B'


Talk To Your Daughter (Album Version) by Robben Ford duration:  4.175665833333333  minutes   segments: 834  songid  b'SOCKRKE12AB018DD02' trackid  b'TRBBOBQ12903CC5186'


You Better Stop by Curtis Mayfield duration:  6.865407333333334  minutes   segments: 1982  songid  b'SOSTESB12A6D4F9A66' trackid  b'TRBBOFH128F14A2A46'


I Got Rhythm by Oscar Peterson Trio duration:  3.095067333333333  minutes   segments: 824  songid  b'SOZVMQJ12AB018D482' trackid  b'TRBBOJM12903CD1BDD'


Hopelessly by Rick Astley duration:  3.5848631666666666  minutes   segments: 643  songid  b'SOFFOFH12A8AE45129' trackid  b'TRBBOUN128F425A4E6'


Jericho  (Straigh

Seize the day by Time Factory duration:  4.074223666666667  minutes   segments: 1090  songid  b'SOKKZIB12A8C13F5DB' trackid  b'TRBBZJP128F4287992'


Ombre Et Lumière by Vincent Bruley duration:  8.306495833333333  minutes   segments: 1086  songid  b'SOFRVYQ12A8C137BAC' trackid  b'TRBBZSO128F4257B30'


Warung Beach by John Digweed duration:  6.751774833333333  minutes   segments: 1375  songid  b'SOUEDBC12AC90972E5' trackid  b'TRBBZJW12903CF71A6'


Mutter Erde by Monika_ Wolfgang duration:  2.9796931666666664  minutes   segments: 508  songid  b'SOVNLPD12AB018F3F9' trackid  b'TRBBZCM12903CDDDA5'


I've got Vilking in Me by Four Tet duration:  2.270033333333333  minutes   segments: 309  songid  b'SOCDXPZ12AB01844FD' trackid  b'TRBBZWC128F93172DA'


2StepN by North Mississippi Allstars duration:  2.7702781666666665  minutes   segments: 715  songid  b'SOZDQHO12AB018512F' trackid  b'TRBBZFO128F932CD27'


Talk About Me by Screamin' Jay Hawkins duration:  2.5094890000000003  minutes   segments:

You'll Need Somebody On Your Bond by Blind Willie Johnson duration:  3.1194481666666665  minutes   segments: 597  songid  b'SOIQVDP12A8C13510A' trackid  b'TRBBFRR128F426CD17'


Loving By Heart by Jeff And Sheri Easter duration:  3.0532713333333334  minutes   segments: 540  songid  b'SOFZGIT12A58A7CD2D' trackid  b'TRBBPXP128F148511C'


Firewater by Joe Grushecky duration:  4.9031761666666664  minutes   segments: 1147  songid  b'SOISJLZ12AC3DF6DE7' trackid  b'TRBBPGU12903CF7C25'


I'll Meet You By The River (Southern Convention Songs Version) by Bill & Gloria Gaither duration:  3.2470128333333337  minutes   segments: 531  songid  b'SOMDWZC12A8C13A065' trackid  b'TRBBPYO128F426DF2A'


No Soy Uno Más by Jose Luis Perales duration:  3.1869311666666666  minutes   segments: 785  songid  b'SORWORW12A6D4FBBE2' trackid  b'TRBBPPB128F14AB242'


Cancion De Amor Eterno by Carlos Vives duration:  1.8816795  minutes   segments: 388  songid  b'SOURXDF12AB0180C6C' trackid  b'TRBBPSX128F93332EE'


Overt

Feriado by Chico Cesar duration:  3.9706046666666666  minutes   segments: 1081  songid  b'SODVLYP12A6D4FCCA3' trackid  b'TRBBDZF128F424BE45'


Seeing Is Believing by Mike And The Mechanics duration:  3.2261148333333334  minutes   segments: 762  songid  b'SOSAOWI12A6D4F7119' trackid  b'TRBBMPH128F1469775'


V by Zeromancer duration:  4.668074  minutes   segments: 848  songid  b'SORGUQZ12AB0184B31' trackid  b'TRBBMDA12903CB688E'


The Lost Name of God by Amorphis duration:  5.534468666666667  minutes   segments: 728  songid  b'SONUOCM12A8C141FF0' trackid  b'TRBBMXI128F428CCD1'


Sweeter Than You by The Four Pennies duration:  2.6096251666666666  minutes   segments: 440  songid  b'SOSGKSM12A6D4F5626' trackid  b'TRBBMTJ128F1477785'


Push It by Salt-N-Pepa duration:  3.4603461666666666  minutes   segments: 805  songid  b'SOWPAXV12A67ADA046' trackid  b'TRBBMHE128EF341D09'


On Tomorrow by Captain Beefheart & The Magic Band duration:  3.442931166666667  minutes   segments: 820  songid  b'SOL

...

A Part Of Me by 40 Grit duration:  4.177842833333333  minutes   segments: 902  songid  b'SOHZWRJ12A8C14081B' trackid  b'TRAKJYQ128F934B22E'


Strange Days ( LP Version ) by The Doors duration:  3.1094346666666666  minutes   segments: 579  songid  b'SOUAGDY12A6D4F63CF' trackid  b'TRAKJOW128F1489DE4'


Nivel De Mafia by Jesus Otero - Luny Tunes duration:  3.1960740000000003  minutes   segments: 929  songid  b'SONBXKE12A81C209B1' trackid  b'TRAKJDO128F4222AED'


Safe Sex by Erick Sermon duration:  3.7246183333333334  minutes   segments: 967  songid  b'SOXNSSY12A6D4F9728' trackid  b'TRAKJTB128F148C560'


Space Station Liberty by Scott Glasgow duration:  3.713734  minutes   segments: 489  songid  b'SOLGYJO12A6BD55664' trackid  b'TRAKJZU128EF34BE38'


Hechizo by Gran Coquivacoa duration:  3.1481830000000004  minutes   segments: 862  songid  b'SOQKZSN12A58A7BB72' trackid  b'TRAKJIN128F92D964B'


London Bridge by Fergie duration:  3.4825503333333336  minutes   segments: 881  songid  b'SOIJEWM1

Two Wheels In the Ditch by Undertakin' Daddies duration:  2.4337340000000003  minutes   segments: 566  songid  b'SOUIWLO12AB017CBC2' trackid  b'TRAKVAS128F92FC385'


Monsters In Tinsel Town by Astrovamps duration:  3.9288088333333335  minutes   segments: 820  songid  b'SOQMIMZ12AB018B499' trackid  b'TRAKVJQ12903CB141D'


I'm Wanderin' by Jackie Wilson duration:  2.6797203333333335  minutes   segments: 483  songid  b'SODBFVM12AB0184A53' trackid  b'TRAKVQI12903CB7343'


Atomic Beat Boy (Album Version) by Helen Love duration:  3.6127271666666667  minutes   segments: 853  songid  b'SOBSOLH12A58A7E673' trackid  b'TRAKVAA128F932B103'


Intro by Papoose duration:  0.7745230000000001  minutes   segments: 167  songid  b'SODSOLK12AB018BB66' trackid  b'TRAKVFJ12903CD4045'


Dime Si Me Quieres by Andy & Lucas duration:  3.5587406666666666  minutes   segments: 972  songid  b'SOXKTIB12A8C14474A' trackid  b'TRAKVHK128F92DCC3C'


La Mémoire Des Jours by Raphaël duration:  4.260999166666667  minutes   

Walkin' In by Tracy Byrd duration:  3.038904  minutes   segments: 744  songid  b'SOMSXGC12A6D4F962D' trackid  b'TRAKCFG128F149D9B7'


Let It Be by The Cate Brothers duration:  3.732890333333333  minutes   segments: 728  songid  b'SOFTSMC12AB017C6BD' trackid  b'TRAKCCA128F92F4EDC'


Mirror (Album) by Cinder duration:  2.7876931666666667  minutes   segments: 465  songid  b'SOJZZVE12A58A7999D' trackid  b'TRAKCUU128F1489FAD'


Viens Danser Jeannette by Jean Segurel duration:  2.868672666666667  minutes   segments: 719  songid  b'SOEFQLO12A8C1362CE' trackid  b'TRAKCGP128F427CAF4'


No me imagino by Fran Perea duration:  3.2369991666666666  minutes   segments: 861  songid  b'SOGKRNP12A81C20860' trackid  b'TRAKCHU128F4234E44'


Bruk Out and Wine by Elephant Man duration:  2.5996115  minutes   segments: 719  songid  b'SOWBVJL12A8AE4720C' trackid  b'TRAKCAT128F4238DC6'


Iki Tas Corba by Mustafa Sandal duration:  3.7942781666666665  minutes   segments: 1113  songid  b'SOEVJRN12A8C135403' tracki

Spritlåten by Ronny Eriksson duration:  2.669706666666667  minutes   segments: 603  songid  b'SOPNKLV12AB018B51E' trackid  b'TRAKLOM12903CC8F8A'


Rocks Off by The Rolling Stones duration:  4.5405095  minutes   segments: 1048  songid  b'SOJDHGS12A6310D85E' trackid  b'TRAKLHO128F42BA554'


Strikes-3 by Atom duration:  7.317761166666667  minutes   segments: 2178  songid  b'SOQHUQW12AB0189F4A' trackid  b'TRAKLSA12903CDA3EB'


Run The Show featuring Don Omar [en Espanol] by Kat DeLuna duration:  3.55787  minutes   segments: 981  songid  b'SOQZLAF12A8AE48030' trackid  b'TRAKLQJ128F42607F9'


Camelot by Richard Burton duration:  2.4881556666666667  minutes   segments: 489  songid  b'SOKZLSY12A8C1320E1' trackid  b'TRAKSRQ128F4269AE8'


Sally Ann by The Cufflinks duration:  2.783339333333333  minutes   segments: 607  songid  b'SOGHWUG12A6D4FB3D3' trackid  b'TRAKSBD128F4249E50'


Bill's Speech - A New Kind of Warrior by Rolfe Kent duration:  1.6718291666666665  minutes   segments: 293  songid  

Warm Spring Night by Claw Hammer duration:  4.389434666666667  minutes   segments: 921  songid  b'SOVKUWM12AC9097662' trackid  b'TRAKANW12903CF8A20'


Newborn Blessing (World) by Sharon Burch duration:  3.230468666666667  minutes   segments: 598  songid  b'SOTMIYY12AAA15FAF8' trackid  b'TRAKAVM128F92F34F2'


I Feel So Good by J.B. Lenoir duration:  1.9104141666666665  minutes   segments: 449  songid  b'SOTNYEO12A6D4F98C3' trackid  b'TRAKABH128F14A0747'


You Don't Know Me by Charlie Rich duration:  4.479121666666667  minutes   segments: 731  songid  b'SODHPGT12A6D4F781D' trackid  b'TRAKAJM128F429BEED'


He's Alright (Album Version) by Fortitude duration:  4.341108166666667  minutes   segments: 1105  songid  b'SOUHUYT12A67021546' trackid  b'TRAKKHN128E07855F9'


In A Distant Place (World) by R. Carlos Nakai duration:  3.9688631666666665  minutes   segments: 753  songid  b'SOJQOCV12AB0183CEA' trackid  b'TRAKKLW128F9317389'


Prolog auf Erden by Equilibrium duration:  3.6506046666666667  

In [3]:
import numpy

from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections
from nearpy.filters import NearestFilter
from nearpy.storage import MemoryStorage
# Dimension of our vector space
dimension = 26
print(segments)
# Create a random binary hash with 10 bits
rbp = RandomBinaryProjections('rbp', 10)

# Create engine with pipeline configuration
engine = Engine(dimension,\
                lshashes=[rbp],\
                vector_filters=[NearestFilter(5)],\
               storage=MemoryStorage())

# Index 1000000 random vectors (set their data to a unique string)
for segment_key in song_segments:
    v = song_segments[segment_key]
    engine.store_vector(v, segment_key)
print("indexed")


{}
indexed


In [4]:
def get_user_song_set(filename):
    user_songs={}
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter='\t')
        for row in csv_reader:
            user_id=row[0]
            song_id=row[1]
            if user_id not in user_songs:
                user_songs[user_id]=set()
            user_songs[user_id].add(song_id)
    return user_songs


In [5]:
user_songs=get_user_song_set("Collaborative-Filtering-Million-Song-Dataset/year1_test_triplets_hidden-test.txt")

In [33]:
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
import numpy as np

def average_precision(actual_songs, predicted_songs, k=10):
    #print(len(predicted_songs),len(actual_songs))
    if len(predicted_songs) > k:
        predicted_songs = predicted_songs[:k]
    
    print(len(predicted_songs),len(actual_songs))
    if not actual_songs:
        return 0.0

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted_songs):
        if p in actual_songs and p not in predicted_songs[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
            print(score, num_hits)
    return score / min(len(actual_songs), k)


def mean_average_precision(predicted_songs, actual_songs, k=10):
    print("map")
    return np.mean([average_precision(a, p, k) for a, p in zip(actual_songs, predicted_songs)])

In [34]:
from random import sample
def get_nearest_songs(engine, song_id):
    song_id=song_id.encode()
    if song_id in song_segments_array:
        pass
    if not (song_id in song_segments_array):
        return []
    song_segments=song_segments_array[song_id]
    song_scores={}
    song_segments = sample(song_segments,3)
    for segment in song_segments:
        near_segments = engine.neighbours(segment)
        for near_segment in near_segments:
            segment_key = near_segment[1]
            song_id = get_song_id_from_segment_key(segment_key)
            if song_id not in song_scores:
                song_scores[song_id]=0
            song_scores[song_id]+=1
    return song_scores

In [38]:
import operator
def get_all_nearest_songs(engine, songs):
    all_nearest_songs = {}
    cnt=0
    print("no of songs for input ",len(songs))
    for song_id in songs:
        #print("song ",cnt)
        ns = get_nearest_songs(engine, song_id)
        for ns_key in ns:
            if ns_key not in all_nearest_songs:
                all_nearest_songs[ns_key]=ns[ns_key]
            else:
                all_nearest_songs[ns_key]+=ns[ns_key]
        cnt+=1
    return all_nearest_songs

def predicted_actual(engine, user_song_set):
    user_predictions = {}
    actual = []
    predicted = []
    for user in user_song_set:
        songs = list(user_song_set[user])
        n =  len(songs)
        songs_in = songs[:(n//2)]
        songs_out = songs[(n//2):]

        nearest_songs = get_all_nearest_songs(engine, songs_in)
        sorted_by_likelihood = sorted(nearest_songs.items(), key=operator.itemgetter(1),reverse=True)
        pred = [a[0] for a in sorted_by_likelihood]
        if len(pred)==0:
            continue
        predicted.append(pred[:n//2])
        actual.append(songs_out)
    return predicted, actual
        


In [39]:
print(len(song_segments))
print("Num of users ",len(user_songs))
predicted, actual = predicted_actual(engine, user_songs)


8577406
Num of users  526
no of songs for input  4
no of songs for input  6
no of songs for input  9
no of songs for input  5
no of songs for input  16
no of songs for input  3
no of songs for input  10
no of songs for input  10
no of songs for input  4
no of songs for input  9
no of songs for input  5
no of songs for input  10
no of songs for input  8
no of songs for input  5
no of songs for input  3
no of songs for input  3
no of songs for input  7
no of songs for input  7
no of songs for input  7
no of songs for input  9
no of songs for input  8
no of songs for input  14
no of songs for input  6
no of songs for input  16
no of songs for input  16
no of songs for input  3
no of songs for input  5
no of songs for input  9
no of songs for input  6
no of songs for input  4
no of songs for input  3
no of songs for input  4
no of songs for input  6
no of songs for input  3
no of songs for input  9
no of songs for input  5
no of songs for input  3
no of songs for input  15
no of songs for 

no of songs for input  10
no of songs for input  3
no of songs for input  3
no of songs for input  3
no of songs for input  12
no of songs for input  6
no of songs for input  7
no of songs for input  4
no of songs for input  4
no of songs for input  4
no of songs for input  6
no of songs for input  3
no of songs for input  4
no of songs for input  3
no of songs for input  10
no of songs for input  3
no of songs for input  18
no of songs for input  2
no of songs for input  4
no of songs for input  4
no of songs for input  3
no of songs for input  6
no of songs for input  11
no of songs for input  10
no of songs for input  7
no of songs for input  13
no of songs for input  2
no of songs for input  4
no of songs for input  5
no of songs for input  13
no of songs for input  8
no of songs for input  3
no of songs for input  10
no of songs for input  6
no of songs for input  14
no of songs for input  11
no of songs for input  3
no of songs for input  11
no of songs for input  3
no of songs f

In [42]:
print(predicted)
print(actual)
print(mean_average_precision(predicted, actual))

[["b'SOBSEGK12A58A7BEBF'", "b'SOJLTEX12AB0184688'", "b'SOWQVVZ12A58A7B65C'", "b'SOXUVWM12AB018BE30'", "b'SOSMURO12A8C13BBFE'", "b'SOHKLCA12A8C1414AB'", "b'SOFQVFL12A6701F8DC'", "b'SOEFLFW12AB018D477'", "b'SOZPTVU12A6D4F8B4D'", "b'SOFUBUF12AC3DF98FF'"], ["b'SORYAVK12A8C13A10E'", "b'SOTUKVB12AB0181477'", "b'SOUALQN12A6D4F98AA'", "b'SOTKYKL12A8AE45B28'", "b'SOBCLZT12AB017F960'", "b'SOKZIXZ12AAF3B2CEC'", "b'SOKHHXO12A8C13D2DF'", "b'SOHUJXE12AB018ADD3'"], ["b'SOHIHSR12AB0184E8F'", "b'SOXPWTF12A81355824'", "b'SOQSYGY12A8C137E0F'", "b'SOGPMNP12AB0186B39'", "b'SOEXIYD12A58A7D818'", "b'SOBBFWL12A8C140087'", "b'SOXMPXX12AB01891E3'", "b'SOFZBXX12AB0181731'", "b'SONPKNE12AB0180390'", "b'SOWVZIQ12A8C1433C9'", "b'SOQZNYW12A8C1411B3'", "b'SOIFTHQ12AF72A4829'"], ["b'SOSIANM12AB018CC80'", "b'SOQQDOT12A8C13BB64'", "b'SORZCXI12AB0183DB4'", "b'SOZOEJC12CF5CFAD99'"], ["b'SOBFTUR12AB0184285'", "b'SOEAASN12A58A7932D'", "b'SOXPTZV12A58A7DF86'", "b'SOUJQRS12A6D4F7ADE'", "b'SOCRCJB12A8C1340CD'", "b'SOYEJKC12A8C

In [27]:
print('SONFOEG12A8C13B8B8'.encode())

b'SONFOEG12A8C13B8B8'
